# Booking (бронирование отелей)
**Цели:** <br>● провести предобработку данных, для последующего их удобного анализа
<br>
● проанализировать данные в соответсвии с имеющимися задачами
<br>
<br>
**Задачи:** <br>● Выяснитьи из каких стран пользовател совершили наибольшее число успешных бронирований?
<br>
● Посчитать количество случаев, когда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type)
<br>
● Узнать на какой месяц чаще всего оформляли бронь в 2016 году и изменился ли он в 2017 году?
<br>
● Понять на какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017?
<br>
● Посчитать сколько клиентов было потеряно по двум группам: клиенты с детьми, клиенты без детей. Иными словами, посчитать метрику под названием Churn Rate.
<br>
<br>
**Входные данные:** <br>входные данные представляют собой файл csv, содержащий в себе информацию о бронировании отелей
<br>
<br>
**Используемые библиотеки:** <br>pandas
<br>numpy
<br>
<br>
Для начала импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np

Считаем файл с данными, который мы предварительно сохранили. Заметим, что вместо запятых, характерных для файлов "csv", там используются ";". Учтем это при считывании. Запишем его в переменную df:

In [2]:
df = pd.read_csv("booking.csv", sep = ";")

Выведем основную информацию о данных:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Hotel                     119390 non-null  object 
 1   Is Canceled               119390 non-null  int64  
 2   Lead Time                 119390 non-null  int64  
 3   arrival full date         119390 non-null  object 
 4   Arrival Date Week Number  119390 non-null  int64  
 5   Stays in Weekend nights   119390 non-null  int64  
 6   Stays in week nights      119390 non-null  int64  
 7   stays total nights        119390 non-null  int64  
 8   Adults                    119390 non-null  int64  
 9   Children                  119386 non-null  float64
 10  Babies                    119390 non-null  int64  
 11  Meal                      119390 non-null  object 
 12  Country                   118902 non-null  object 
 13  Reserved Room Type        119390 non-null  o

Описание данных:
<br>● Hotel – тип отеля (City Hotel или Resort Hotel)
<br>● Is canceled – бронирование было отменено (1) или нет (0); неотменённое считается успешным
<br>● Lead time – количество дней, прошедших между датой бронирования и датой прибытия
<br>● Arrival full date – полная дата прибытия
<br>● Arrival date week number – номер недели прибытия
<br>● Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
<br>● Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
<br>● Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
<br>● Adults – число взрослых
<br>● Children – число детей
<br>●Babies – число младенцев
<br>● Meal – выбранный тип питания
<br>● Country – страна происхождения клиента
<br>● Reserved room type – тип зарезервированного номера
<br>● Assigned room type – тип полученного номера (может отличаться от забронированного)
<br>● Customer type – тип бронирования
<br>● Reservation status – значение последнего статуса брони: Canceled – было отменено клиентом; Check-Out – клиент зарегистрировался, но уже покинул отель; No-Show – клиент не зарегистрировался и сообщил администрации отеля причину
<br>● Reservation status date – дата обновления статуса

Всего 119390 строк и 18 столбцов, при этом в некоторых столбцах существуют пропуски. Обратим внимание на типы данных, но вернемся к этому вопросу после удаления всего лишнего.


In [4]:
df.head()

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Week Number,Stays in Weekend nights,Stays in week nights,stays total nights,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,01.07.2015,27,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,01.07.2015
1,Resort Hotel,0,737,01.07.2015,27,0,0,0,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,01.07.2015
2,Resort Hotel,0,7,01.07.2015,27,0,1,1,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,02.07.2015
3,Resort Hotel,0,13,01.07.2015,27,0,1,1,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,02.07.2015
4,Resort Hotel,0,14,01.07.2015,27,0,2,2,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,03.07.2015


Названия столбцов написаны не в стиле snake_case. Исправим это:

In [5]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

In [6]:
df.duplicated().sum()

36235

In [7]:
df = df.drop_duplicates()

Теперь поймем, что для наших задач некоторые столбцы не нужны. Удалим их:

In [8]:
 df = df.drop(['lead_time',
       'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', "meal",
       'customer_type', "reservation_status", "reservation_status_date"], axis=1)
df.head()

,hotel,is_canceled,arrival_full_date,children,babies,country,reserved_room_type,assigned_room_type
0,Resort Hotel,0,01.07.2015,0.0,0,PRT,C,C
1,Resort Hotel,0,01.07.2015,0.0,0,PRT,C,C
2,Resort Hotel,0,01.07.2015,0.0,0,GBR,A,C
3,Resort Hotel,0,01.07.2015,0.0,0,GBR,A,A
4,Resort Hotel,0,01.07.2015,0.0,0,GBR,A,A


Проверим на количество пропусков:

In [9]:
df.isna().mean()

hotel                 0.000000
is_canceled           0.000000
arrival_full_date     0.000000
children              0.000048
babies                0.000000
country               0.005207
reserved_room_type    0.000000
assigned_room_type    0.000000
dtype: float64

Проценты пропусков среди тех столбцов, где они есть, везде маленькие, заменим их на другие значения, чтобы не было ошибки во время анализа

In [10]:
df["children"] = df["children"].fillna(0)
df["country"] = df["country"].fillna("No")
df.isna().mean()

hotel                 0.0
is_canceled           0.0
arrival_full_date     0.0
children              0.0
babies                0.0
country               0.0
reserved_room_type    0.0
assigned_room_type    0.0
dtype: float64

Теперь пропусков нет. 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83155 entries, 0 to 119389
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   hotel               83155 non-null  object 
 1   is_canceled         83155 non-null  int64  
 2   arrival_full_date   83155 non-null  object 
 3   children            83155 non-null  float64
 4   babies              83155 non-null  int64  
 5   country             83155 non-null  object 
 6   reserved_room_type  83155 non-null  object 
 7   assigned_room_type  83155 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 5.7+ MB


Нужно изменить типы данных в некоторых столбцах. Так например столбец "children" имеет тип float

In [12]:
df["children"] = df["children"].astype(int)

Еще для удобства я переименую значения в первом столбце. "Resort Hotel" заменю на "R", а "City Hotel" на "С"

In [13]:
df["hotel"] = df["hotel"].apply(lambda x: x[0])
df.head()

,hotel,is_canceled,arrival_full_date,children,babies,country,reserved_room_type,assigned_room_type
0,R,0,01.07.2015,0,0,PRT,C,C
1,R,0,01.07.2015,0,0,PRT,C,C
2,R,0,01.07.2015,0,0,GBR,A,C
3,R,0,01.07.2015,0,0,GBR,A,A
4,R,0,01.07.2015,0,0,GBR,A,A


Проверим уникальные значений по каждому столбцу, чтобы убедиться, что все данные имеют одинаковый вид.

In [14]:
df['hotel'].sort_values().unique()

array(['C', 'R'], dtype=object)

In [15]:
df['is_canceled'].sort_values().unique()

array([0, 1])

In [16]:
df['arrival_full_date'].sort_values().unique()

array(['01.01.2016', '01.01.2017', '01.02.2016', '01.02.2017',
       '01.03.2016', '01.03.2017', '01.04.2016', '01.04.2017',
       '01.05.2016', '01.05.2017', '01.06.2016', '01.06.2017',
       '01.07.2015', '01.07.2016', '01.07.2017', '01.08.2015',
       '01.08.2016', '01.08.2017', '01.09.2015', '01.09.2016',
       '01.10.2015', '01.10.2016', '01.11.2015', '01.11.2016',
       '01.12.2015', '01.12.2016', '02.01.2016', '02.01.2017',
       '02.02.2016', '02.02.2017', '02.03.2016', '02.03.2017',
       '02.04.2016', '02.04.2017', '02.05.2016', '02.05.2017',
       '02.06.2016', '02.06.2017', '02.07.2015', '02.07.2016',
       '02.07.2017', '02.08.2015', '02.08.2016', '02.08.2017',
       '02.09.2015', '02.09.2016', '02.10.2015', '02.10.2016',
       '02.11.2015', '02.11.2016', '02.12.2015', '02.12.2016',
       '03.01.2016', '03.01.2017', '03.02.2016', '03.02.2017',
       '03.03.2016', '03.03.2017', '03.04.2016', '03.04.2017',
       '03.05.2016', '03.05.2017', '03.06.2016', '03.06

In [17]:
df['children'].sort_values().unique()

array([ 0,  1,  2,  3, 10])

In [18]:
df['babies'].sort_values().unique()

array([ 0,  1,  2,  9, 10])

In [19]:
df['country'].sort_values().unique()

array(['ABW', 'AGO', 'AIA', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
       'ATA', 'ATF', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA',
       'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BOL', 'BRA', 'BRB',
       'BWA', 'CAF', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'CN', 'COL',
       'COM', 'CPV', 'CRI', 'CUB', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI',
       'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ESP', 'EST', 'ETH',
       'FIN', 'FJI', 'FRA', 'FRO', 'GAB', 'GBR', 'GEO', 'GGY', 'GHA',
       'GIB', 'GLP', 'GNB', 'GRC', 'GTM', 'GUY', 'HKG', 'HND', 'HRV',
       'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR',
       'ITA', 'JAM', 'JEY', 'JOR', 'JPN', 'KAZ', 'KEN', 'KHM', 'KIR',
       'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBY', 'LCA', 'LIE', 'LKA',
       'LTU', 'LUX', 'LVA', 'MAC', 'MAR', 'MCO', 'MDG', 'MDV', 'MEX',
       'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MOZ', 'MRT', 'MUS', 'MWI',
       'MYS', 'MYT', 'NAM', 'NCL', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL',
       'NZL', 'No', '

In [20]:
df['reserved_room_type'].sort_values().unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'L', 'P'], dtype=object)

In [21]:
df['assigned_room_type'].sort_values().unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'P'],
      dtype=object)

Все данные в каждом столбце имеют одинаковый вид. Перейдем к выполнению задач.
<br>
<br>
## Выяснитьи из каких стран пользовател совершили наибольшее число успешных бронирований?

Заменим значения с столбце "is_canceled" для удобства дальнейшей работы. Теперь 0 будет заменен на -1, т.е. если отмены брони не было, тогда это будет обозначено "-1"

In [22]:
df.loc[:, 'is_canceled'] = df.loc[:, 'is_canceled'].replace(0, -1)

Создадим df1, к котрому применим фильтр, обозначающий, что бронь отменена не была.

In [23]:
df1 = df.query('is_canceled == -1')
df1.head()

,hotel,is_canceled,arrival_full_date,children,babies,country,reserved_room_type,assigned_room_type
0,R,-1,01.07.2015,0,0,PRT,C,C
1,R,-1,01.07.2015,0,0,PRT,C,C
2,R,-1,01.07.2015,0,0,GBR,A,C
3,R,-1,01.07.2015,0,0,GBR,A,A
4,R,-1,01.07.2015,0,0,GBR,A,A


Теперь создадим сводную таблицу, в которой посчитаем сумму значений столбца "is_canceled" для каждой страны

In [24]:
df2 = pd.pivot_table(df1,
               values=["is_canceled"],
               index =["country"],
               aggfunc=[np.sum])


Отсортируем таблицу по значениям столбца "('sum', "is_canceled")"

In [25]:
df2 = df2.sort_values([('sum', "is_canceled")])

Таким образом наибольшие по модулю значение в столбце "('sum', "is_canceled")" будут обозночать страну, где было совершено наибольшее количество успешных бронирований. Создадим топ 10:

In [26]:
df2.head(10)

,sum
,is_canceled
country,
PRT,-16138
GBR,-7803
FRA,-6808
ESP,-5076
DEU,-4130
IRL,-2215
ITA,-1887
BEL,-1616


Получается:
<br>● Португалия - 16138
<br>● Великобритания - 7803
<br>● Франция - 6808
<br>● Испания - 5076
<br>● Германия - 4130
<br>● Ирландия - 2215
<br>● Италия - 1887
<br>● Бельгия - 1616
<br>● Нидерладны - 1512
<br>● США - 1370

## Посчитать количество случаев, когда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type)

Создадим столбец "matching", в который запишем значение **True**, если ячейка столбца "reserved_room_type" равна ячейки столбца "assigned_room_type", иначе **False **

In [27]:
df["matching"] = df.apply(lambda x: x["reserved_room_type"] == x["assigned_room_type"], axis=1)
df.head(10)

,hotel,is_canceled,arrival_full_date,children,babies,country,reserved_room_type,assigned_room_type,matching
0,R,-1,01.07.2015,0,0,PRT,C,C,True
1,R,-1,01.07.2015,0,0,PRT,C,C,True
2,R,-1,01.07.2015,0,0,GBR,A,C,False
3,R,-1,01.07.2015,0,0,GBR,A,A,True
4,R,-1,01.07.2015,0,0,GBR,A,A,True
6,R,-1,01.07.2015,0,0,PRT,C,C,True
7,R,-1,01.07.2015,0,0,PRT,C,C,True
8,R,1,01.07.2015,0,0,PRT,A,A,True
9,R,1,01.07.2015,0,0,PRT,D,D,True
10,R,1,01.07.2015,0,0,PRT,E,E,True


Посчитаем количетсво значений "False" в этом столбце:

In [28]:
df['matching'].value_counts()

True     70832
False    12323
Name: matching, dtype: int64

Таких случаев было 12323. 
<br>
<br>
## Узнать на какой месяц чаще всего оформляли бронь в 2016 году и изменился ли он в 2017 году?

Разобьем имеющиеся даты из столбца "arrival_full_date" на год, день, месяц. Запишем эти значения в отдельные столбцы, зададим им тип **int**

In [29]:
df["year"] = df["arrival_full_date"].apply(lambda x: x.split(".")[-1])
df["month"] = df["arrival_full_date"].apply(lambda x: x.split(".")[-2])
df["day"] = df["arrival_full_date"].apply(lambda x: x.split(".")[0])
df["year"] = df["year"].astype(int)
df["month"] = df["month"].astype(int)
df["day"] = df["day"].astype(int)


Примерим фильтр, указав, что год далжен быть 2015, а бронь не отменена. 

In [30]:
df2 = df.query('year == 2015 and is_canceled == -1')
df2.head()

,hotel,is_canceled,arrival_full_date,children,babies,country,reserved_room_type,assigned_room_type,matching,year,month,day
0,R,-1,01.07.2015,0,0,PRT,C,C,True,2015,7,1
1,R,-1,01.07.2015,0,0,PRT,C,C,True,2015,7,1
2,R,-1,01.07.2015,0,0,GBR,A,C,False,2015,7,1
3,R,-1,01.07.2015,0,0,GBR,A,A,True,2015,7,1
4,R,-1,01.07.2015,0,0,GBR,A,A,True,2015,7,1


Посчитаем сколько значений каждого месяца в колонке "month"

In [31]:
df2['month'].value_counts()

9     2153
10    2090
8     1786
12    1498
11    1285
7     1119
Name: month, dtype: int64

9 месяц - больше всего успешных бронирований.
<br>
Теперь то же самое сделаем с фильтром на 2016 год.

In [32]:
df3 = df.query('year == 2016 and is_canceled == -1')
df3['month'].value_counts()

8     2857
10    2781
3     2749
5     2646
7     2642
4     2585
9     2551
6     2459
11    2379
2     2098
12    2006
1     1407
Name: month, dtype: int64

Видим, что 8 месяц является наиболее популярным по успешным бронированиям в 2016 году. Это отличается от 2017 года. Проверим теперь не только по бронированиям, которые не были отменены, а по всем броням:

In [33]:
df3 = df.query('year == 2015')
df3['month'].value_counts()

9     2697
10    2521
8     2344
12    1856
7     1612
11    1525
Name: month, dtype: int64

In [34]:
df3 = df.query('year == 2016')
df3['month'].value_counts()

8     4279
10    3960
7     3709
3     3617
9     3613
4     3580
5     3569
6     3358
11    3180
12    2998
2     2610
1     1699
Name: month, dtype: int64

Теперь ситуация поменялась, за 2015 год самый популярный месяц - 9, а в 2016 8. Таким образом месяц изменился.

## Понять на какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017?

Применим филльтр, в котором укажем, что тип отеля должен быть "City Hotel" и бронь должная быть отменеа:

In [35]:
df3 = df.query('hotel == "C" and is_canceled == 1')
df3.head()

,hotel,is_canceled,arrival_full_date,children,babies,country,reserved_room_type,assigned_room_type,matching,year,month,day
40061,C,1,01.07.2015,0,0,PRT,A,A,True,2015,7,1
40062,C,1,01.07.2015,0,0,PRT,A,A,True,2015,7,1
40063,C,1,01.07.2015,0,0,PRT,A,A,True,2015,7,1
40064,C,1,02.07.2015,0,0,PRT,A,A,True,2015,7,2
40065,C,1,02.07.2015,0,0,PRT,A,A,True,2015,7,2


Применим фильтр, в котром год должен быть 2015 и посчитаем в новой таблице значения каждого месяца.

In [36]:
df4 = df3.query('year == 2015')
df4["month"].value_counts()

9     299
10    284
8     220
7     218
12    211
11    136
Name: month, dtype: int64

Наибольшее значение - тот месяц, в котором бронирование отменялось чаще всего. По данным видно, что это 9 месяц (сентябрь).
<br>
Теперь сделаем аналогично с 2016 годом

In [37]:
df5 = df3.query('year == 2016')
df5["month"].value_counts()

8     898
10    828
12    752
9     746
4     737
7     672
3     636
5     634
6     608
11    599
2     363
1     205
Name: month, dtype: int64

Самый популярный месяц по отмене бронирования для этого типа отеля - 8 (август). И посчитаем то же самое для 2017 года:

In [38]:
df6 = df3.query('year == 2017')
df6["month"].value_counts()

5    1088
4     988
7     977
8     960
6     902
3     715
2     599
1     543
Name: month, dtype: int64

Больше всего отмен бронирования пришлось на 5 месяц (май).
<br>
## Посчитать сколько клиентов было потеряно по двум группам: клиенты с детьми, клиенты без детей. Иными словами, посчитать метрику под названием Churn Rate.

Создадим отдельный столбик "kids", в котором укажем, есть ли дети или нет (значения True/False)

In [39]:
def count_children(x):
    ans = x["children"] + x["babies"]
    return ans != 0


df["kids"] = df.apply(count_children, axis=1)
df.head(15)

,hotel,is_canceled,arrival_full_date,children,babies,country,reserved_room_type,assigned_room_type,matching,year,month,day,kids
0,R,-1,01.07.2015,0,0,PRT,C,C,True,2015,7,1,False
1,R,-1,01.07.2015,0,0,PRT,C,C,True,2015,7,1,False
2,R,-1,01.07.2015,0,0,GBR,A,C,False,2015,7,1,False
3,R,-1,01.07.2015,0,0,GBR,A,A,True,2015,7,1,False
4,R,-1,01.07.2015,0,0,GBR,A,A,True,2015,7,1,False
6,R,-1,01.07.2015,0,0,PRT,C,C,True,2015,7,1,False
7,R,-1,01.07.2015,0,0,PRT,C,C,True,2015,7,1,False
8,R,1,01.07.2015,0,0,PRT,A,A,True,2015,7,1,False
9,R,1,01.07.2015,0,0,PRT,D,D,True,2015,7,1,False
10,R,1,01.07.2015,0,0,PRT,E,E,True,2015,7,1,False


Теперь к новому столбику применим фильтр, в котором укажем, что детей нет. Посчитаем количество отмененных и успешных бронирований. 

In [40]:
df7 = df.query("kids == False")
a = df7["is_canceled"].value_counts()
print(a)

-1    53598
 1    20561
Name: is_canceled, dtype: int64


Посчитаем какой процент от всех составлял отток:

In [41]:
a[1] / sum(a) * 100

27.72556264243045

27.7 процентов людей без детьми отменили бронирование. Посчитаем так же сколько людей с детьми отменили бронирование:

In [42]:
df8 = df.query("kids == True")
b = df8["is_canceled"].value_counts()
print(b)

-1    5902
 1    3094
Name: is_canceled, dtype: int64


In [43]:
b[1] / sum(b) * 100

34.39306358381503

34.4 процентов. Среди людей с детьми этот процент выше. 
<br>
## Вывод
Итак, в процессе предобработки данных мне удалось избавиться от лишних столбцов, данные из которых не нужны были для решения поставленных задач. Также я проверила на наличие явных и неявных дубликатов. Помимо этого я  заменила пропущенные ячейки. Я создала несколько столбцов, данные в которых были продуктом анализа имеющихся столбцов. 
<br>
<br>
Я решила все имеющиеся задачи, что значит, что я провела анализ имеющихся данных, подробно описав каждый свой шаг.